# Pipeline configuration

In [84]:
confi_dict = {

    'general':{
        'root_dir':r"C:\Users\na-ch\Desktop\estudio\Master_Big_Data\03_TFM\02_Code\dataset_test",
        'dataset_dir': "CASIA_IrisV1"
    },

    '1_tratarDataset': {
        'show_first' : False
    },

    '1.1_dataAugmentation':{
        'gaussianNoise' : False,
        'stdGN': [5, 10, 15, 20, 25, 30],
        'skipFiles': 1, 
        ''' 
        1 means that you are skiping 1 for the noise augmentation, 2 skiping 2 and so on. If you don't want to skip files set it as 0
        '''
        'afinTransformation': True
    },

    '2.1_segmentation':{
        'redNeuronal' : "Iris_unet_d5.h5"
    }

}

# 1: Tratar el dataset CASIA



## tratar_dataset_casia()

In [25]:
def tratar_dataset_casia(dict):
    
    print("Función 1, tratar_dataset_casia()")
    
    ''' 
    Importando librerías
    '''
    try:
        import os
    except:
        !pip install os
        import os
    try:
        import shutil
    except:
        !pip install shutil
        import shutil
    try:
         import imageio
    except:
        !pip install imageio
        import imageio
    try:
        import matplotlib.pyplot as plt
    except:
        !pip install matplotlib
        import matplotlib.pyplot as plt
    ''' 
    Estableciendo la root de la función
    '''
    os.chdir(dict['general']['root_dir'])
    
    def copy_all_samples(path_samples, destination_directory):
        '''
        Copia todas las muestras del dataset de CASIA a un único directorio.
        '''
        if not os.path.exists(path_samples):
            print("El directorio indicado como input no existe")
        elif not os.path.exists(destination_directory):
            ''' 
            Comprobamos que el directorio del output no existe
            '''
            os.mkdir(destination_directory) # Creamos un nuevo directorio 
            for root, dirs, files in os.walk(path_samples):
                for file in files:
                    path_file = os.path.join(root,file)
                    shutil.copy2(path_file,destination_directory)   
            print("-> Muestras copiadas satisfactoriamente.")
        elif len(os.listdir(destination_directory)) < 1:
            ''' 
            Comprobamos que el directorio del output no está vacío
            '''
            for root, dirs, files in os.walk(path_samples):
                for file in files:
                    path_file = os.path.join(root,file)
                    shutil.copy2(path_file,destination_directory)   
            print("-> Muestras copiadas satisfactoriamente.")
        else:
            print("-> Muetras previamente copiadas.")

    copy_all_samples(dict['general']['dataset_dir'], dict['general']['dataset_dir']+"_Unificado")

    if dict['1_tratarDataset']['show_first']:
        ''' 
        Si en la configuración show_first es True, se mostrará el primer elemento del nuevo directorio.
        '''
        path_img = dict['general']['dataset_dir']+"_Unificado" + os.sep + os.listdir(dict['general']['dataset_dir']+"_Unificado")[0]
        img = imageio.imread(path_img)
        plt.title("Primer elemento de " + dict['general']['dataset_dir']+"_Unificado")
        plt.imshow(img)
    
    ''' 
    Pasando el diccionario con la configuración a la siguiente función
    '''
    return dict


## 1.1: Data Augmentation
data_augmentation()

In [80]:
def data_augmentation(dict):

    print("Función 1.1, data_augmentation()")
    
    ''' 
    Importando librerías
    '''
    try:
        import os
    except:
        !pip install os
        import os
    try:
        import imageio
    except:
        !pip install imageio
        import imageio
    try:
        import matplotlib.pyplot as plt
    except:
        !pip install matplotlib
        import matplotlib.pyplot as plt
    try:
        from tensorflow.keras.layers import GaussianNoise
    except:
        !pip install tensorflow
        from tensorflow.keras.layers import GaussianNoise
    try:
        import numpy as np
    except:
        !pip install numpy
        import numpy as np
    import random

    ''' 
    Estableciendo la root de la función
    '''
    os.chdir(dict['general']['root_dir'])
    
    ''' 
    Comprobamos si el ruido gaussiano ha sido establecido como True en la configuración
    '''

    if dict['1.1_dataAugmentation']['gaussianNoise']:

        ''' 
        Comprobamos si el ruido gaussiano se ha aplicado anteriormente, si es así, el primer elemento del directorio debería de tener el sufijo _augmentation
        '''
        if "_augment" in os.listdir(dict['general']['dataset_dir'] + "_Unificado")[1]:
            print("El ruido gaussiano ya se ha ejecutado con anterioridad")
        else:     
            ''' 
            Desde el directorio unificado creamos nuevas imágenes utilizando el ruido gaussiano como data augmentation.
            Añadimos un contador para que el augmentation sea de 1 cada dos imágenes.
            '''
            i = 0
            for filename in os.listdir(dict['general']['dataset_dir'] + "_Unificado"):
                if dict['1.1_dataAugmentation']['skipFiles'] == 0:
                    path = dict['general']['dataset_dir'] + "_Unificado" + os.sep + filename
                    image = imageio.imread(path)/255
                    std_rdm = random.choice(dict['1.1_dataAugmentation']['stdGN'])
                    gaussean_function = GaussianNoise(std_rdm/100)
                    noisey = gaussean_function(image.astype(np.float32),training=True)
                    name = path[:-4] + '_augment.bmp'
                    plt.imsave(name, noisey, cmap='gray')
                elif i < dict['1.1_dataAugmentation']['skipFiles']:
                    path = dict['general']['dataset_dir'] + "_Unificado" + os.sep + filename
                    image = imageio.imread(path)/255
                    std_rdm = random.choice(dict['1.1_dataAugmentation']['stdGN'])
                    gaussean_function = GaussianNoise(std_rdm/100)
                    noisey = gaussean_function(image.astype(np.float32),training=True)
                    name = path[:-4] + '_augment.bmp'
                    plt.imsave(name, noisey, cmap='gray')
                    i += 1
                else:
                    i = 0
    else:
        print("Saltando el ruido gausseano en data augmentation")

    ''' 
    Pasando el diccionario con la configuración a la siguiente función
    '''
    return dict

# 2 Red U-Net v2

## 2.1 segmentation()

In [ ]:
def segmentation(dict):

    print("Función 2.1, segmentation()")

    ''' 
    Importando librerías
    '''
    try:
        import os
    except:
        !pip install os
        import os
    ''' 
    Estableciendo la root de la función
    '''
    os.chdir(dict['general']['root_dir'])
    

## 2.2 normalization

In [ ]:
def normalization(dict):

    print("Función 2.2, normalization()")

    ''' 
    Importando librerías
    '''
    try:
        import os
    except:
        !pip install os
        import os
    ''' 
    Estableciendo la root de la función
    '''
    os.chdir(dict['general']['root_dir'])

    

# Definir y ejecutar el pipeline

## Definición del pipeline

In [4]:
try:
    from sklearn.preprocessing import FunctionTransformer
    from sklearn.pipeline import Pipeline
except ImportError as e:
    !pip install sklearn
    from sklearn.preprocessing import FunctionTransformer
    from sklearn.pipeline import Pipeline

In [81]:
_1_tratar_dataset_pip = FunctionTransformer(tratar_dataset_casia)
_1_1_data_augmentation_pip = FunctionTransformer(data_augmentation)

In [82]:
iris_recognition_pipeline = Pipeline([('_1_tratarDataset', _1_tratar_dataset_pip), ('_1_1_dataAugmentation', _1_1_data_augmentation_pip)])


## Ejecición pipeline


In [85]:
iris_recognition_pipeline.transform(confi_dict)

Función 1, tratar_dataset_casia()
-> Muetras previamente copiadas.
Función 1.1, data_augmentation()
Saltando el ruido gausseano en data augmentation


{'general': {'root_dir': 'C:\\Users\\na-ch\\Desktop\\estudio\\Master_Big_Data\\03_TFM\\02_Code\\dataset_test',
  'dataset_dir': 'CASIA_IrisV1'},
 '1_tratarDataset': {'show_first': False},
 '1.1_dataAugmentation': {'gaussianNoise': False,
  'stdGN': [5, 10, 15, 20, 25, 30],
  'skipFiles': 1,
  " \n        1 means that you are skiping 1 for the noise augmentation, 2 skiping 2 and so on. If you don't want to skip files set it as 0\n        afinTransformation": True}}